### TODO

Bu notebook sipariş ve envanter verilerinde teklif, iade ve ret nedeni dolu olan satırları filtrelenir. Ardından değişen parça dönüşümleri iki veri setine de uygulanarak export alınır. Tüm yıllara ait bizimle paylaşılmış olan verileri içermektedir.

1) Düzenleme adımı olarak OTOBB ve CANTER in ayrı değerlendirilmesi istendiği için değişen malzeme kısmı tekrardan düzenlenmelidir

In [2]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.width', None)

### 0) Sipariş Verisi Hazırlama

Bu kısımdaki tüm işlemler neler yapıldığını göstermek amaçlıdır. Bu başlık sonucunda oluşturulmuş veri "31_08_teklif_iade_temiz_all_columns.csv" ismiyle kaydedilmiştir. Bu nedenle 1. başlıktan devam edilebilir.

In [16]:
df = pd.read_csv("2_siparis_data_merged_22_07 (1).csv")

C:\Users\Asus\AppData\Local\Temp\ipykernel_14444\2030908777.py:1: DtypeWarning: Columns (20,28,29,34) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("2_siparis_data_merged_22_07 (1).csv")


In [17]:
df

,Üretim yeri,Satış belgesi,Yaratma tarihi,İki basamaklı no.,Saat,Satış organizasyonu,Dağıtım kanalı,Bölüm,Kalem,Müşteri,Ad,Malzeme,Malzeme kısa metni,İlk Malzeme,Kaç defa değişti?,Fiyat grubu,Sipariş miktarı,Teyit edilen küm.mkt,Durum,Geliş Trh,Tahsis Tarihi,Tahsis Saati,İlk Dönem,Son Dönem,Unnamed: 24,Orta. Bul,Genel Bul.,Servis Drc,Son Tahsis Tarihi,Hesap Tarihi,Hesap Saati,Hesap Miktari,Satış belgesi türü,Sipariş nedeni,Ret nedeni,Araç tipi,Tedarik yeri,Sipariş Kriteri,Plnl.tsl.süresi,Tedarik Türü,Kayar ortalama fiyat,ÜY'ye özgü mlz.drm.,Sonraki SD belgesi
0,5000,1002763582,2018-06-30,6,03:06:52 PM,OTOB,BS,OY,50,800000,AKYÜZLER OTOMOTİV DIŞ,LF16519,YAĞ FİLTRESİ,LF16015VE,1,10.0,2.0,2.0,Normal,6/30/2018,NaN,12:00:00 AM,6.2018,6.2025,0,0.0,0.0,0.0,NaN,NaN,12:00:00 AM,0.0,WTEK,901,NaN,AVENUE E6,CUMMINS,B,0,I,0,NaN,1.107911e+09
1,5000,1002763573,2018-06-30,6,11:59:12 AM,OTOB,BS,OY,350,600371,ÖZPAM OTOMOTİV PETROL LTD. ŞTİ.,5572-LGE-01Y,"GELİŞTİRİLMİŞ TİP AKÜ 225Ah, TEMSA LOGO",NaN,0,10.0,1.0,1.0,Normal,6/30/2018,NaN,12:00:00 AM,6.2018,6.2025,0,0.0,0.0,0.0,NaN,NaN,12:00:00 AM,0.0,WTEK,901,NaN,SAFARI-HD,İNCİ AKÜ,T,0,Y,0,NaN,1.107911e+09
2,5000,1002763574,2018-06-30,6,12:19:33 PM,OTOB,BS,OY,10,600241,BOLU AS OTOMOTİV TUR. NAK.,MX902755,"KELEPÇE, INTERCOOLER",NaN,0,10.0,6.0,1.0,Normal,6/30/2018,NaN,12:00:00 AM,6.2018,6.2025,0,0.0,0.0,0.0,NaN,NaN,12:00:00 AM,0.0,WTEK,901,NaN,PRESTIJ E6,MFTBC,V,0,I,0,NaN,1.107911e+09
3,5000,1002763575,2018-06-30,6,12:21:33 PM,OTOB,BS,OY,10,600241,BOLU AS OTOMOTİV TUR. NAK.,TM187142Y,HAVA KOMPRESORU,NaN,0,10.0,2.0,0.0,Normal,6/30/2018,NaN,12:00:00 AM,6.2018,6.2025,0,0.0,0.0,0.0,NaN,NaN,12:00:00 AM,0.0,WTEK,901,NaN,PRESTIJ E6,FILKAR,K,0,Y,0,NaN,1.107911e+09
4,5000,1002763576,2018-06-30,6,12:23:54 PM,OTOB,BS,OY,10,600030,BAYRAKTAR OTOMOTİV TİC VE SAN.LTD.,5668-LGP-01,"ARKA BLOK FAR,SAG,DG2010-8B,YENI",HD253129,1,10.0,1.0,1.0,Normal,6/30/2018,NaN,12:00:00 AM,6.2018,6.2025,0,0.0,0.0,0.0,NaN,NaN,12:00:00 AM,0.0,WTEK,901,NaN,MARATON,DANYANG,P,0,I,0,NaN,1.107911e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
396495,4000,1108064087,2025-06-30,6,13:50:46,MA00,YS,PY,440,601063,KAPANLAR OTOMOTİV SAN. VE TİC. A.Ş.,ML239124Y,"E6 PRESTIJ FILTRE,YAKIT",NaN,0,1.0,50.0,50.0,Normal,2025-06-30 00:00:00,2025-06-30,13:51:26,6.2018,6.2025,0,1.0,1.0,1.0,2025-06-30,2025-06-30,13:51:26,50.0,YPSP,40,NaN,PRESTIJ E6,KRON FİLT,I,0,Y,0,NaN,NaN
396496,4000,1108064087,2025-06-30,6,13:50:46,MA00,YS,PY,430,601063,KAPANLAR OTOMOTİV SAN. VE TİC. A.Ş.,MK580528AY,"LAMBA KOMPLE,DONUS SINYALI,ON,SAG",NaN,0,1.0,10.0,10.0,Normal,2025-06-30 00:00:00,2025-06-30,13:51:26,6.2018,6.2025,0,1.0,1.0,1.0,2025-06-30,2025-06-30,13:51:26,10.0,YPSP,40,NaN,FEB91E,ÇITAK,M,0,Y,0,NaN,NaN
396497,4000,1108064087,2025-06-30,6,13:50:46,MA00,YS,PY,420,601063,KAPANLAR OTOMOTİV SAN. VE TİC. A.Ş.,MK580527AY,"FAR COMPLESI,DONUS SINYALI,ON,SOL",NaN,0,1.0,10.0,10.0,Normal,2025-06-30 00:00:00,2025-06-30,13:51:26,6.2018,6.2025,0,1.0,1.0,1.0,2025-06-30,2025-06-30,13:51:26,10.0,YPSP,40,NaN,FEB91E,ÇITAK,M,0,Y,0,NaN,NaN
396498,4000,1108064087,2025-06-30,6,13:50:49,MA00,YS,PY,490,601063,KAPANLAR OTOMOTİV SAN. VE TİC. A.Ş.,5802114243,SILINDIR KAPAGI-E6,NaN,0,1.0,2.0,0.0,Normal,2025-06-30 00:00:00,2025-06-30,13:51:26,6.2018,6.2025,0,0.0,0.0,0.0,NaN,2025-06-30,13:51:26,0.0,YPSP,40,NaN,CANTER S-E,FPT,H,0,I,0,NaN,NaN


In [3]:
df["Yaratma tarihi"] = pd.to_datetime(df["Yaratma tarihi"], errors="coerce")

In [18]:
# Red nedenleri sözlüğü
red_nedenleri = {
    "01":  "TESLIMAT TERMINI COK GEC",
    "02":  "KOTU KALITE",
    "03":  "COK PAHALI",
    "04":  "RAKIP DAHA IYI",
    "05":  "GARANTI",
    "10": "MAKUL OLMAYAN MUSTERI TALEBI",
    "11": "MUSTERIYE EK TESLIMAT YAPILIYOR",
    "20": "LOST SALE",
    "21": "TAKAS",
    "22": "PAHALI SERVIS VE YEDEK PARCA",
    "25": "ANLASMALI IHALE",
    "27": "MUSTERIYLE ZAMANINDA TEMAS SAGLANAMAMASI",
    "28": "TESVIKLE ILGILI SEBEPLER",
    "29": "FORWARDED (EU)",
    "31": "KABUL-S",
    "32": "RED",
    32:"RED",
    "33": "RED-B",
    "34": "RED-F",
    "35": "RED-M",
    "36": "Zaman Aşımından Reddedildi",
    "37": "Tedarik Edilememektedir",
    "40": "Siparişe Dönüştürüldü",
    "50": "İşlem kontrol ediliyor",
    "51": "Fiy-Mkt Borç Dekont Talebi İptali",
    "52": "Direkt Tedarikçiden Gönderim (SB-SSH)",
    "Z2": "MTO-MAHSUPLAŞTI",
    "Z3": "MTO-SİPARİŞ İPTALİ",
    "Z5": "Müşteri tarafından iptal edildi",
    "Z6": "Temsa Almanya'dan gönderiliyor"
}
df["Ret nedeni"] = df["Ret nedeni"].replace(red_nedenleri)
## Alındığı doküman: "https://docs.google.com/document/d/1iXYbunAk5i_QFvx_EE2Bpn7G-mHcH_5E/edit?usp=drive_link&ouid=103315257096636267674&rtpof=true&sd=true"


In [19]:
siparis_nedeni_teklif = {
    "008": "(TKLF) WEB'den GELEN - Back Order-NORMAL",
    "011": "(TKLF) NORMAL",
    "012": "(TKLF) IHALE",
    "013": "(TKLF) WEB'den GELEN",
    "014": "(TKLF) WEB'den GELEN - Lost Sale",
    "015": "(TKLF) WEB'den GELEN - Back Order-ACİL",
    "016": "(TKLF) WEB'den GELEN -Lost S.& Back Ord.",
    "040": "Teklif Değerlendirme",
}

siparis_nedeni_iade = {
    # Resmi "İADE =" kodları
    "025": "İADE=KATALOG HATASI",
    "026": "İADE=HATALI SİPARİŞ KARŞILAMA",
    "027": "İADE=HATALI SİPARİŞ",
    "028": "İADE=HATALI PARÇA (MİKTAR /PARÇA NO)",
    "029": "İADE=DENEMEK İÇİN İSTENEN PARÇA",
    "02A": "İADE = YANLIŞ BİLDİRİLEN PARÇA",
    "02B": "İADE = PARÇA İYİLEŞTİRMESİ",
    "02C": "İADE = STOK GENÇLEŞTİRME",
    "02D": "İADE = BAYİ DEN İADE ALIM",
    "02E": "İADE = FİYAT FARKI",
    "02F": "GEÇMİŞ YILLARDAN İADE",
    "02H": "İADE = TRANSİT PARÇA",
    "02I": "İADE = BAYİDEN STOK İÇİN İADE ALIM",
    "02K": "İADE = MÜCBİR SEBEP İLE İADE ALIMI",

    # Mantıken iade sayılabilecekler
    "101": "Parça uyumsuzluğu ve fonksiyonel hatalar",
    "102": "Parçanın hasarlı teslimatı",
    "103": "Miktar farkı",
    "104": "Bozuk mal",
    "035": "Tedarikçi Claim İadesi",
}
replace_map = {**siparis_nedeni_teklif, **siparis_nedeni_iade}
df["Sipariş nedeni"] = df["Sipariş nedeni"].replace(replace_map)
df["Sipariş nedeni"].value_counts()


Sipariş nedeni
901                                   340812
900                                    20086
YP3                                     6093
YP0                                     3706
YP1                                     1702
YP5                                     1021
İADE = BAYİ DEN İADE ALIM                439
B53                                      365
I17                                      348
İADE = TRANSİT PARÇA                     346
B30                                      252
40                                       241
İADE = MÜCBİR SEBEP İLE İADE ALIMI       125
902                                       49
DDS                                       41
903                                       40
28                                        31
BZ5                                       13
17                                         7
İADE = BAYİDEN STOK İÇİN İADE ALIM         7
29                                         4
B55                                     

In [20]:
teklif_kodlari = ["TKLF", "MTEK", "PBLT", "PTBE", "PTEK", "STEK", "VTEK", "WKAM", "WPAK", "WTEK", "ZTKL"]

In [21]:
siparis_teklif_kodlari = set(siparis_nedeni_teklif.values())
siparis_iade_kodlari = set(siparis_nedeni_iade.values())


filtered_df = df[
    (df["Ret nedeni"].isna()) &                                   # Ret nedeni boş
    (~df["Satış belgesi türü"].isin(teklif_kodlari + ["IADE"])) & # Satış belgesi türü teklif veya iade değil
    (~df["Sipariş nedeni"].isin(siparis_teklif_kodlari)) &         # Sipariş nedeni teklif değil
    (~df["Sipariş nedeni"].isin(siparis_iade_kodlari))             # Sipariş nedeni iade değil
].copy()

In [22]:
filtered_df["Satış belgesi türü"].value_counts()

Satış belgesi türü
YPSP    158408
YPAS      9373
ZBED       371
YPDS         3
Name: count, dtype: int64

In [24]:
df

,Üretim yeri,Satış belgesi,Yaratma tarihi,İki basamaklı no.,Saat,Satış organizasyonu,Dağıtım kanalı,Bölüm,Kalem,Müşteri,Ad,Malzeme,Malzeme kısa metni,İlk Malzeme,Kaç defa değişti?,Fiyat grubu,Sipariş miktarı,Teyit edilen küm.mkt,Durum,Geliş Trh,Tahsis Tarihi,Tahsis Saati,İlk Dönem,Son Dönem,Unnamed: 24,Orta. Bul,Genel Bul.,Servis Drc,Son Tahsis Tarihi,Hesap Tarihi,Hesap Saati,Hesap Miktari,Satış belgesi türü,Sipariş nedeni,Ret nedeni,Araç tipi,Tedarik yeri,Sipariş Kriteri,Plnl.tsl.süresi,Tedarik Türü,Kayar ortalama fiyat,ÜY'ye özgü mlz.drm.,Sonraki SD belgesi
0,5000,1002763582,2018-06-30,6,03:06:52 PM,OTOB,BS,OY,50,800000,AKYÜZLER OTOMOTİV DIŞ,LF16519,YAĞ FİLTRESİ,LF16015VE,1,10.0,2.0,2.0,Normal,6/30/2018,NaN,12:00:00 AM,6.2018,6.2025,0,0.0,0.0,0.0,NaN,NaN,12:00:00 AM,0.0,WTEK,901,NaN,AVENUE E6,CUMMINS,B,0,I,0,NaN,1.107911e+09
1,5000,1002763573,2018-06-30,6,11:59:12 AM,OTOB,BS,OY,350,600371,ÖZPAM OTOMOTİV PETROL LTD. ŞTİ.,5572-LGE-01Y,"GELİŞTİRİLMİŞ TİP AKÜ 225Ah, TEMSA LOGO",NaN,0,10.0,1.0,1.0,Normal,6/30/2018,NaN,12:00:00 AM,6.2018,6.2025,0,0.0,0.0,0.0,NaN,NaN,12:00:00 AM,0.0,WTEK,901,NaN,SAFARI-HD,İNCİ AKÜ,T,0,Y,0,NaN,1.107911e+09
2,5000,1002763574,2018-06-30,6,12:19:33 PM,OTOB,BS,OY,10,600241,BOLU AS OTOMOTİV TUR. NAK.,MX902755,"KELEPÇE, INTERCOOLER",NaN,0,10.0,6.0,1.0,Normal,6/30/2018,NaN,12:00:00 AM,6.2018,6.2025,0,0.0,0.0,0.0,NaN,NaN,12:00:00 AM,0.0,WTEK,901,NaN,PRESTIJ E6,MFTBC,V,0,I,0,NaN,1.107911e+09
3,5000,1002763575,2018-06-30,6,12:21:33 PM,OTOB,BS,OY,10,600241,BOLU AS OTOMOTİV TUR. NAK.,TM187142Y,HAVA KOMPRESORU,NaN,0,10.0,2.0,0.0,Normal,6/30/2018,NaN,12:00:00 AM,6.2018,6.2025,0,0.0,0.0,0.0,NaN,NaN,12:00:00 AM,0.0,WTEK,901,NaN,PRESTIJ E6,FILKAR,K,0,Y,0,NaN,1.107911e+09
4,5000,1002763576,2018-06-30,6,12:23:54 PM,OTOB,BS,OY,10,600030,BAYRAKTAR OTOMOTİV TİC VE SAN.LTD.,5668-LGP-01,"ARKA BLOK FAR,SAG,DG2010-8B,YENI",HD253129,1,10.0,1.0,1.0,Normal,6/30/2018,NaN,12:00:00 AM,6.2018,6.2025,0,0.0,0.0,0.0,NaN,NaN,12:00:00 AM,0.0,WTEK,901,NaN,MARATON,DANYANG,P,0,I,0,NaN,1.107911e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
396495,4000,1108064087,2025-06-30,6,13:50:46,MA00,YS,PY,440,601063,KAPANLAR OTOMOTİV SAN. VE TİC. A.Ş.,ML239124Y,"E6 PRESTIJ FILTRE,YAKIT",NaN,0,1.0,50.0,50.0,Normal,2025-06-30 00:00:00,2025-06-30,13:51:26,6.2018,6.2025,0,1.0,1.0,1.0,2025-06-30,2025-06-30,13:51:26,50.0,YPSP,40,NaN,PRESTIJ E6,KRON FİLT,I,0,Y,0,NaN,NaN
396496,4000,1108064087,2025-06-30,6,13:50:46,MA00,YS,PY,430,601063,KAPANLAR OTOMOTİV SAN. VE TİC. A.Ş.,MK580528AY,"LAMBA KOMPLE,DONUS SINYALI,ON,SAG",NaN,0,1.0,10.0,10.0,Normal,2025-06-30 00:00:00,2025-06-30,13:51:26,6.2018,6.2025,0,1.0,1.0,1.0,2025-06-30,2025-06-30,13:51:26,10.0,YPSP,40,NaN,FEB91E,ÇITAK,M,0,Y,0,NaN,NaN
396497,4000,1108064087,2025-06-30,6,13:50:46,MA00,YS,PY,420,601063,KAPANLAR OTOMOTİV SAN. VE TİC. A.Ş.,MK580527AY,"FAR COMPLESI,DONUS SINYALI,ON,SOL",NaN,0,1.0,10.0,10.0,Normal,2025-06-30 00:00:00,2025-06-30,13:51:26,6.2018,6.2025,0,1.0,1.0,1.0,2025-06-30,2025-06-30,13:51:26,10.0,YPSP,40,NaN,FEB91E,ÇITAK,M,0,Y,0,NaN,NaN
396498,4000,1108064087,2025-06-30,6,13:50:49,MA00,YS,PY,490,601063,KAPANLAR OTOMOTİV SAN. VE TİC. A.Ş.,5802114243,SILINDIR KAPAGI-E6,NaN,0,1.0,2.0,0.0,Normal,2025-06-30 00:00:00,2025-06-30,13:51:26,6.2018,6.2025,0,0.0,0.0,0.0,NaN,2025-06-30,13:51:26,0.0,YPSP,40,NaN,CANTER S-E,FPT,H,0,I,0,NaN,NaN


In [23]:
filtered_df["Üretim yeri"].value_counts()

Üretim yeri
4000    168155
Name: count, dtype: int64

In [10]:
filtered_df.to_csv("csv_files/31_08_teklif_iade_temiz_all_columns.csv", index=False)

In [11]:
filtered_df

,Üretim yeri,Satış belgesi,Yaratma tarihi,İki basamaklı no.,Saat,Satış organizasyonu,Dağıtım kanalı,Bölüm,Kalem,Müşteri,Ad,Malzeme,Malzeme kısa metni,İlk Malzeme,Kaç defa değişti?,Fiyat grubu,Sipariş miktarı,Teyit edilen küm.mkt,Durum,Geliş Trh,Tahsis Tarihi,Tahsis Saati,İlk Dönem,Son Dönem,Unnamed: 24,Orta. Bul,Genel Bul.,Servis Drc,Son Tahsis Tarihi,Hesap Tarihi,Hesap Saati,Hesap Miktari,Satış belgesi türü,Sipariş nedeni,Ret nedeni,Araç tipi,Tedarik yeri,Sipariş Kriteri,Plnl.tsl.süresi,Tedarik Türü,Kayar ortalama fiyat,ÜY'ye özgü mlz.drm.,Sonraki SD belgesi
27,5000,1107911074,2018-06-30,6,21:10:11,OTOB,BS,OY,20,601013,HACI MUSTAFA BAĞIŞLAR,D1895550,EGZOZ BORUSU CONTASI,NaN,0,10.0,1.0,1.0,Normal,2018-07-02,2018-06-30,21:10:27,6.2018,6.2025,0,1.0,1.0,1.0,2018-06-30,2018-06-30,21:10:27,1.0,YPSP,901,NaN,SAFIR V E6,DAF,X,0,I,0,NaN,NaN
28,5000,1107911074,2018-06-30,6,20:58:34,OTOB,BS,OY,10,601013,HACI MUSTAFA BAĞIŞLAR,D1895549,EKSOZ CONTASI,NaN,0,10.0,1.0,1.0,Normal,2018-07-02,2018-06-30,21:10:27,6.2018,6.2025,0,1.0,1.0,1.0,2018-06-30,2018-06-30,21:10:27,1.0,YPSP,901,NaN,SAFIR V E6,DAF,X,0,I,0,NaN,NaN
29,5000,1107911074,2018-06-30,6,21:10:11,OTOB,BS,OY,60,601013,HACI MUSTAFA BAĞIŞLAR,D1923003,"HAVA GIRIS CONTASI,TURBO",NaN,0,10.0,1.0,1.0,Normal,2018-07-02,2018-06-30,21:10:27,6.2018,6.2025,0,1.0,1.0,1.0,2018-06-30,2018-06-30,21:10:27,1.0,YPSP,901,NaN,MARATON E6,DAF,O,0,I,0,NaN,NaN
34,5000,1107911074,2018-06-30,6,21:10:11,OTOB,BS,OY,30,601013,HACI MUSTAFA BAĞIŞLAR,D2187464,EGZOZ MANIFOLD CONTASI,D1917011,1,10.0,6.0,6.0,Normal,2018-07-02,2018-06-30,21:10:27,6.2018,6.2025,0,1.0,1.0,1.0,2018-06-30,2018-06-30,21:10:27,6.0,YPSP,901,NaN,HD RHD,DAF,P,0,I,0,NaN,NaN
40,5000,1107911074,2018-06-30,6,21:10:11,OTOB,BS,OY,40,601013,HACI MUSTAFA BAĞIŞLAR,D2199692,"EGZOZ MANIFOLDU,MOTOR",D1919888,1,10.0,1.0,1.0,Normal,2018-07-02,2018-06-30,21:10:27,6.2018,6.2025,0,1.0,1.0,1.0,2018-06-30,2018-06-30,21:10:27,1.0,YPSP,901,NaN,MARATON E6,DAF,P,0,I,0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1229697,5000,1106065475,2025-06-30,6,08:14:22,OTOB,XD,OY,90,238,TFR SAS,1627-LPE-01Y,"BRAKET, AVARE KASNAK",NaN,0,10.0,1.0,1.0,Normal,2025-06-30,2025-06-30,08:15:42,6.2018,6.2025,0,1.0,1.0,1.0,2025-06-30,2025-06-30,08:15:42,1.0,ZIHY,901,NaN,AVENUE E6,TURKERLER,X,0,Y,0,NaN,NaN
1229698,5000,1106065475,2025-06-30,6,08:14:22,OTOB,XD,OY,101,238,TFR SAS,275531-10580,"DEGISTIRME KABLOSU,L 10580mm",NaN,0,10.0,1.0,0.0,Normal,2025-06-30,2025-06-30,08:15:42,6.2018,6.2025,0,0.0,0.0,0.0,NaN,2025-06-30,08:15:42,0.0,ZIHY,901,NaN,TOURMA E5C,KONGSBERG,K,0,I,0,NaN,NaN
1229699,5000,1106065475,2025-06-30,6,08:14:22,OTOB,XD,OY,111,238,TFR SAS,3503-ANE-01Y,"KÖRÜK,ARKA FREN, (23.216.16.14)",NaN,0,10.0,1.0,0.0,Normal,2025-06-30,2025-06-30,08:15:42,6.2018,6.2025,0,0.0,0.0,0.0,NaN,2025-06-30,08:15:42,0.0,ZIHY,901,NaN,MD 7 E6C,ARFESAN,Y,0,Y,0,NaN,NaN
1229700,5000,1106065475,2025-06-30,6,08:14:22,OTOB,XD,OY,120,238,TFR SAS,3724-ANH-01Y,"DIREKSIYON CEKME KOLU,ON AKS",NaN,0,10.0,2.0,2.0,Normal,2025-06-30,2025-06-30,08:15:42,6.2018,6.2025,0,1.0,1.0,1.0,2025-06-30,2025-06-30,08:15:42,2.0,ZIHY,901,NaN,MD 7 E6FPT,DITAS,K,0,Y,0,NaN,NaN


### 1) Değişen Parça Uygulaması

In [7]:
e=pd.read_excel(r"Partvision Envanter 14.08.25 1.xlsx")

In [9]:
e["Üretim Yeri"].value_counts()

Üretim Yeri
4000    15389
Name: count, dtype: int64

In [3]:
excel_path = r"Partvision Envanter 14.08.25 1.xlsx"
sheets_dict = pd.read_excel(excel_path, sheet_name=None)

df_list = []
for sheet_name, df in sheets_dict.items():
    df["Üretim yeri"] = sheet_name
    df_list.append(df)

df_envanter = pd.concat(df_list, ignore_index=True)

In [12]:
df_envanter


,Malzeme Kodu,Malzeme Tanımı,Değişen Malzeme Kodu,Yerli Malzeme Kodu,Muadil Malzeme Kodu,Toplam Stok Miktarı,Kullanılabilir stok adedi,Back order adedi,"Açık, stok sip.adt.","Açık, acil sip.adt.",Tedarik türü,Tedarik yeri,Araç tipi,Standart maliyet,Ortalama maliyet,Yeni Sprş Seviyesi,Azami Stok,Mip Parti Byklğ,Mip Karakteristiği,Plnlı Tslmat Srsi,Sabitleme Srsi,Lead Time,Özel Tdrk,Blokaj,Parça Açıklaması,Replacibility Kod,Değişen Parça Replacibility,Orjinal Parca no,Orj. Parca Stok Ad.,Sipariş Kriteri,Cluster,Üst Sınıf,Ana Grup,Min. Order Qty.,Min. Package Qty.,Mevsimsel,Yurtiçi Rabat,Yurtdışı Rabat,Malzeme Tanımı ( EN ),Üretim Yeri,Key Cluster Kodu,Üst Sınıf Kodu,Ana Grup Kodu,VOR,Teps,Phase Out,Üretim yeri
0,0123525501,ALTERNATÖR 100 A,0123525501Y,0123525501Y,NaN,0.0,0.0,0,0.0,0,I,BOSCH-TRY,OPALIN9 E4,337.49,0.00,0,0,42,PD,60,NaN,63,NaN,TB,NaN,NaN,O,NaN,0.0,P,ALTERNATÖR,MOTOR DETAYLARI,MOTOR,NaN,NaN,M3,7.0,D,"ALTERNATOR, 100 A",4000,Y0010,U001,G01,NaN,X,NaN,4000 canter
1,0123525501Y,ALTERNATOR 100 A,0123525507,NaN,NaN,0.0,0.0,0,0.0,0,Y,BOSCH-TRY,OPALIN9 E5,0.01,0.00,0,0,42,PD,45,NaN,41,NaN,NaN,NaN,O,O,NaN,0.0,P,ALTERNATÖR,MOTOR DETAYLARI,MOTOR,NaN,NaN,M3,5.0,C,ALTERNATOR 100 A,4000,Y0010,U001,G01,NaN,X,NaN,4000 canter
2,0123525507,ALTERNATOR 100 A,NaN,NaN,NaN,0.0,0.0,0,0.0,0,Y,BOSCH-TRY,METROP E4M,5497.60,0.00,0,0,42,PD,60,NaN,63,NaN,NaN,NaN,O,NaN,51.26101-7265,0.0,P,ALTERNATÖR,MOTOR DETAYLARI,MOTOR,NaN,NaN,M3,6.0,C,NaN,4000,Y0010,U001,G01,NaN,X,NaN,4000 canter
3,0124525565,ALTERNATOR,NaN,NaN,NaN,0.0,0.0,0,1.0,0,Y,BOSCH-TRY,DIGER,8616.89,7290.58,0,0,42,PD,60,NaN,63,NaN,NaN,"* Otobüste Yaz dönemi sıkıntı yaşanan parçadır, dikkat!!",O,NaN,NaN,0.0,P,ALTERNATÖR,MOTOR DETAYLARI,MOTOR,NaN,NaN,M3,6.0,C,NaN,4000,Y0010,U001,G01,X,X,NaN,4000 canter
4,0178-XC-11-14-01_A,DIN935 M14X2 A2 PASLANMAZ YARIKLI SOMUN,NaN,NaN,NaN,0.0,0.0,0,0.0,0,Y,ANADOL CİV,EQUAD,213.34,0.00,0,0,NaN,NaN,0,NaN,32,NaN,NaN,NaN,NaN,NaN,NaN,0.0,P,SOMUN,HARDWARE,DİĞER,NaN,NaN,NaN,NaN,L,DIN935 M14X2 A2 STAINLESS SLOT NUT,4000,Y0333,U112,G27,NaN,NaN,NaN,4000 canter
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116996,ZF-ECOFLUID M-02E,ŞANZIMAN YAĞ OTOMATİK,NaN,NaN,NaN,0.0,0.0,0,0.0,0,Y,TEMSANEW,SARF,0.00,0.00,0,0,42,PD,75,NaN,63,NaN,NaN,NaN,NaN,NaN,NaN,0.0,P,YAĞ,BAKIM PARÇALARI,BAKIM PAKETLERİ,NaN,NaN,M1,NaN,NaN,TRANSMISSION OIL AUTOMATIC,5000,Y0390,U154,G41,NaN,NaN,NaN,5000 canter
116997,ZIMBA TELI,"ZIMBA TELİ (39X26X36,5cm)",NaN,NaN,NaN,0.0,0.0,0,0.0,0,Y,EMRE END.,DIGER,1.00,0.00,0,0,42,PD,49,3.0,63,NaN,CA,NaN,NaN,NaN,NaN,0.0,P,SON 10 YIL İÇERİSİNDE HAREKET ETMEYEN,SON 10 YIL İÇERİSİNDE HAREKET ETMEYEN,DİĞER,NaN,NaN,M2,5,C,NaN,5000,Y0245,U169,G27,NaN,NaN,NaN,5000 canter
116998,ZIMBA.TELI,"ZIMBA TELİ (39X26X36,5cm)",NaN,NaN,NaN,0.0,0.0,0,0.0,0,Y,PIYASA,DIGER,190.00,270.00,0,0,42,PD,49,NaN,63,NaN,NaN,NaN,NaN,NaN,NaN,0.0,P,SARF,SARF,DİĞER,NaN,NaN,M1,4,L,STABLES,5000,Y0446,U122,G27,NaN,NaN,NaN,5000 canter
116999,ZR905Z,MOTOR SANTRIFUJ YAG FILTRESI,NaN,NaN,NaN,11.0,11.0,0,36.0,0,I,MANNHUMMEL,SAFIR P E5,585.95,471.36,0,0,42,PD,70,NaN,63,NaN,NaN,* Garantiden çıkan Safirlerin bakımında talep görüyor,NaN,NaN,D1922496,1.0,I,FİLTRE,MOTOR DETAYLARI,MOTOR,12,NaN,M3,4,B,ENGINE CENTRIFUGAL OIL FILTER,5000,Y0120,U001,G01,NaN,X,NaN,5000 canter


In [13]:
df_envanter.to_excel("envanter_veri.xlsx")

In [11]:
df_envanter["Üretim yeri"].value_counts()

Üretim yeri
5000 canter    101612
4000 canter     15389
Name: count, dtype: int64

In [26]:
import pandas as pd
import time

start_time = time.time()
print("⏳ Zincir algoritması başladı...")

df_sub = df_envanter[["Malzeme Kodu", "Değişen Malzeme Kodu", "Değişen Parça Replacibility"]].copy()
print(f"Step 1: Alt DataFrame hazır {df_sub.shape}, elapsed={time.time()-start_time:.2f} sn")

df_o = df_sub[df_sub["Değişen Parça Replacibility"] == "O"]
print(f"Step 2: 'O' filtrelendi {df_o.shape}, elapsed={time.time()-start_time:.2f} sn")

next_map = dict(zip(df_o["Malzeme Kodu"].astype(str),
                    df_o["Değişen Malzeme Kodu"].astype(str)))
prev_map = {v: k for k, v in next_map.items()}
print(f"Step 3: Mapping kuruldu (next_map={len(next_map)}, prev_map={len(prev_map)}), elapsed={time.time()-start_time:.2f} sn")

self_loops = [(k, v) for k, v in next_map.items() if k == v]
if self_loops:
    print(f"⚠️ Self-loop bulundu! Toplam {len(self_loops)} adet")
else:
    print("✅ Self-loop bulunmadı")

def detect_cycles(next_map, max_depth=1000):
    cycles = []
    skip = set()
    for start in next_map.keys():
        visited = []
        code = start
        depth = 0
        while code in next_map and depth < max_depth:
            if code in visited:
                cycle_start_index = visited.index(code)
                cycle_path = visited[cycle_start_index:] + [code]
                cycles.append(cycle_path)
                skip.update(cycle_path)
                break
            visited.append(code)
            code = next_map[code]
            depth += 1
    return cycles, skip

cycles, skip_codes = detect_cycles(next_map)
if cycles:
    print(f"⚠️ {len(cycles)} cycle bulundu. İlk 3 örnek:")
else:
    print("✅ Cycle bulunmadı")

def find_root(code):
    while code in prev_map and code not in skip_codes:
        code = prev_map[code]
    return code

def walk_chain(root, max_depth=1000):
    chain = []
    visited = set()
    depth = 0
    while root not in visited and depth < max_depth:
        visited.add(root)
        chain.append(root)
        if root not in next_map or root in skip_codes:
            break
        root = next_map[root]
        depth += 1
    return chain

zincir_map = {}
all_codes = (set(next_map.keys()) | set(next_map.values())) - skip_codes
print(f"Step 4: Zincir kurulumu başlayacak (toplam {len(all_codes)} kod)")

for i, code in enumerate(all_codes, 1):
    root = find_root(code)
    chain = walk_chain(root)
    for c in chain:
        if c not in zincir_map or len(chain) > len(zincir_map[c]):
            zincir_map[c] = chain
    if i % 3000 == 0 or i == len(all_codes):
        print(f"  ...{i}/{len(all_codes)} kod işlendi "
              f"({i/len(all_codes):.1%}), elapsed={time.time()-start_time:.2f} sn")

print(f"Step 5: Zincirler tamamlandı (zincir_map={len(zincir_map)}), elapsed={time.time()-start_time:.2f} sn")

codes = df_envanter["Malzeme Kodu"].astype(str).to_numpy()
result = [str(zincir_map.get(c)) for c in codes]

df_envanter = df_envanter.copy()
df_envanter.loc[:, "Degisen Malzeme List"] = result

print(f"Final shape={df_envanter.shape}, total time={time.time()-start_time:.2f} sn")


⏳ Zincir algoritması başladı...
Step 1: Alt DataFrame hazır (117001, 3), elapsed=0.01 sn
Step 2: 'O' filtrelendi (5576, 3), elapsed=0.01 sn
Step 3: Mapping kuruldu (next_map=5100, prev_map=4749), elapsed=0.01 sn
⚠️ Self-loop bulundu! Toplam 1 adet
⚠️ 3 cycle bulundu. İlk 3 örnek:
Step 4: Zincir kurulumu başlayacak (toplam 9319 kod)
  ...3000/9319 kod işlendi (32.2%), elapsed=0.02 sn
  ...6000/9319 kod işlendi (64.4%), elapsed=0.02 sn
  ...9000/9319 kod işlendi (96.6%), elapsed=0.02 sn
  ...9319/9319 kod işlendi (100.0%), elapsed=0.02 sn
Step 5: Zincirler tamamlandı (zincir_map=9319), elapsed=0.02 sn
Final shape=(117001, 48), total time=0.12 sn


In [27]:
df_envanter

,Malzeme Kodu,Malzeme Tanımı,Değişen Malzeme Kodu,Yerli Malzeme Kodu,Muadil Malzeme Kodu,Toplam Stok Miktarı,Kullanılabilir stok adedi,Back order adedi,"Açık, stok sip.adt.","Açık, acil sip.adt.",Tedarik türü,Tedarik yeri,Araç tipi,Standart maliyet,Ortalama maliyet,Yeni Sprş Seviyesi,Azami Stok,Mip Parti Byklğ,Mip Karakteristiği,Plnlı Tslmat Srsi,Sabitleme Srsi,Lead Time,Özel Tdrk,Blokaj,Parça Açıklaması,Replacibility Kod,Değişen Parça Replacibility,Orjinal Parca no,Orj. Parca Stok Ad.,Sipariş Kriteri,Cluster,Üst Sınıf,Ana Grup,Min. Order Qty.,Min. Package Qty.,Mevsimsel,Yurtiçi Rabat,Yurtdışı Rabat,Malzeme Tanımı ( EN ),Üretim Yeri,Key Cluster Kodu,Üst Sınıf Kodu,Ana Grup Kodu,VOR,Teps,Phase Out,Üretim yeri,Degisen Malzeme List
0,0123525501,ALTERNATÖR 100 A,0123525501Y,0123525501Y,NaN,0.0,0.0,0,0.0,0,I,BOSCH-TRY,OPALIN9 E4,337.49,0.00,0,0,42,PD,60,NaN,63,NaN,TB,NaN,NaN,O,NaN,0.0,P,ALTERNATÖR,MOTOR DETAYLARI,MOTOR,NaN,NaN,M3,7.0,D,"ALTERNATOR, 100 A",4000,Y0010,U001,G01,NaN,X,NaN,4000 canter,"['0123525501', '0123525501Y', '0123525507']"
1,0123525501Y,ALTERNATOR 100 A,0123525507,NaN,NaN,0.0,0.0,0,0.0,0,Y,BOSCH-TRY,OPALIN9 E5,0.01,0.00,0,0,42,PD,45,NaN,41,NaN,NaN,NaN,O,O,NaN,0.0,P,ALTERNATÖR,MOTOR DETAYLARI,MOTOR,NaN,NaN,M3,5.0,C,ALTERNATOR 100 A,4000,Y0010,U001,G01,NaN,X,NaN,4000 canter,"['0123525501', '0123525501Y', '0123525507']"
2,0123525507,ALTERNATOR 100 A,NaN,NaN,NaN,0.0,0.0,0,0.0,0,Y,BOSCH-TRY,METROP E4M,5497.60,0.00,0,0,42,PD,60,NaN,63,NaN,NaN,NaN,O,NaN,51.26101-7265,0.0,P,ALTERNATÖR,MOTOR DETAYLARI,MOTOR,NaN,NaN,M3,6.0,C,NaN,4000,Y0010,U001,G01,NaN,X,NaN,4000 canter,"['0123525501', '0123525501Y', '0123525507']"
3,0124525565,ALTERNATOR,NaN,NaN,NaN,0.0,0.0,0,1.0,0,Y,BOSCH-TRY,DIGER,8616.89,7290.58,0,0,42,PD,60,NaN,63,NaN,NaN,"* Otobüste Yaz dönemi sıkıntı yaşanan parçadır, dikkat!!",O,NaN,NaN,0.0,P,ALTERNATÖR,MOTOR DETAYLARI,MOTOR,NaN,NaN,M3,6.0,C,NaN,4000,Y0010,U001,G01,X,X,NaN,4000 canter,"['0124525216', '0124525565']"
4,0178-XC-11-14-01_A,DIN935 M14X2 A2 PASLANMAZ YARIKLI SOMUN,NaN,NaN,NaN,0.0,0.0,0,0.0,0,Y,ANADOL CİV,EQUAD,213.34,0.00,0,0,NaN,NaN,0,NaN,32,NaN,NaN,NaN,NaN,NaN,NaN,0.0,P,SOMUN,HARDWARE,DİĞER,NaN,NaN,NaN,NaN,L,DIN935 M14X2 A2 STAINLESS SLOT NUT,4000,Y0333,U112,G27,NaN,NaN,NaN,4000 canter,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116996,ZF-ECOFLUID M-02E,ŞANZIMAN YAĞ OTOMATİK,NaN,NaN,NaN,0.0,0.0,0,0.0,0,Y,TEMSANEW,SARF,0.00,0.00,0,0,42,PD,75,NaN,63,NaN,NaN,NaN,NaN,NaN,NaN,0.0,P,YAĞ,BAKIM PARÇALARI,BAKIM PAKETLERİ,NaN,NaN,M1,NaN,NaN,TRANSMISSION OIL AUTOMATIC,5000,Y0390,U154,G41,NaN,NaN,NaN,5000 canter,None
116997,ZIMBA TELI,"ZIMBA TELİ (39X26X36,5cm)",NaN,NaN,NaN,0.0,0.0,0,0.0,0,Y,EMRE END.,DIGER,1.00,0.00,0,0,42,PD,49,3.0,63,NaN,CA,NaN,NaN,NaN,NaN,0.0,P,SON 10 YIL İÇERİSİNDE HAREKET ETMEYEN,SON 10 YIL İÇERİSİNDE HAREKET ETMEYEN,DİĞER,NaN,NaN,M2,5,C,NaN,5000,Y0245,U169,G27,NaN,NaN,NaN,5000 canter,None
116998,ZIMBA.TELI,"ZIMBA TELİ (39X26X36,5cm)",NaN,NaN,NaN,0.0,0.0,0,0.0,0,Y,PIYASA,DIGER,190.00,270.00,0,0,42,PD,49,NaN,63,NaN,NaN,NaN,NaN,NaN,NaN,0.0,P,SARF,SARF,DİĞER,NaN,NaN,M1,4,L,STABLES,5000,Y0446,U122,G27,NaN,NaN,NaN,5000 canter,None
116999,ZR905Z,MOTOR SANTRIFUJ YAG FILTRESI,NaN,NaN,NaN,11.0,11.0,0,36.0,0,I,MANNHUMMEL,SAFIR P E5,585.95,471.36,0,0,42,PD,70,NaN,63,NaN,NaN,* Garantiden çıkan Safirlerin bakımında talep görüyor,NaN,NaN,D1922496,1.0,I,FİLTRE,MOTOR DETAYLARI,MOTOR,12,NaN,M3,4,B,ENGINE CENTRIFUGAL OIL FILTER,5000,Y0120,U001,G01,NaN,X,NaN,5000 canter,None


In [31]:
df_envanter[df_envanter["Malzeme Kodu"] == "RF-623-7232"]

,Malzeme Kodu,Malzeme Tanımı,Değişen Malzeme Kodu,Yerli Malzeme Kodu,Muadil Malzeme Kodu,Toplam Stok Miktarı,Kullanılabilir stok adedi,Back order adedi,"Açık, stok sip.adt.","Açık, acil sip.adt.",Tedarik türü,Tedarik yeri,Araç tipi,Standart maliyet,Ortalama maliyet,Yeni Sprş Seviyesi,Azami Stok,Mip Parti Byklğ,Mip Karakteristiği,Plnlı Tslmat Srsi,Sabitleme Srsi,Lead Time,Özel Tdrk,Blokaj,Parça Açıklaması,Replacibility Kod,Değişen Parça Replacibility,Orjinal Parca no,Orj. Parca Stok Ad.,Sipariş Kriteri,Cluster,Üst Sınıf,Ana Grup,Min. Order Qty.,Min. Package Qty.,Mevsimsel,Yurtiçi Rabat,Yurtdışı Rabat,Malzeme Tanımı ( EN ),Üretim Yeri,Key Cluster Kodu,Üst Sınıf Kodu,Ana Grup Kodu,VOR,Teps,Phase Out,Üretim yeri,Degisen Malzeme List
94435,RF-623-7232,"Raufoss Push-in male M12 / Tube 5/16""",RP311610,RF-623-7232Y,NaN,0.0,0.0,0,0.0,0,Y,TEMSANEW,TS35 EPA15,19.6,0.0,0,0,42,PD,70,3.0,63,NaN,TB,NaN,O,O,NaN,0.0,P,REKOR,HARDWARE,DİĞER,NaN,NaN,M1,6,E,"Raufoss Push-in male M12 / Tube 5/16""",5000,Y0308,U112,G27,NaN,NaN,NaN,5000 canter,None


In [32]:
df_envanter[df_envanter["Malzeme Kodu"] == "RP311610"]

,Malzeme Kodu,Malzeme Tanımı,Değişen Malzeme Kodu,Yerli Malzeme Kodu,Muadil Malzeme Kodu,Toplam Stok Miktarı,Kullanılabilir stok adedi,Back order adedi,"Açık, stok sip.adt.","Açık, acil sip.adt.",Tedarik türü,Tedarik yeri,Araç tipi,Standart maliyet,Ortalama maliyet,Yeni Sprş Seviyesi,Azami Stok,Mip Parti Byklğ,Mip Karakteristiği,Plnlı Tslmat Srsi,Sabitleme Srsi,Lead Time,Özel Tdrk,Blokaj,Parça Açıklaması,Replacibility Kod,Değişen Parça Replacibility,Orjinal Parca no,Orj. Parca Stok Ad.,Sipariş Kriteri,Cluster,Üst Sınıf,Ana Grup,Min. Order Qty.,Min. Package Qty.,Mevsimsel,Yurtiçi Rabat,Yurtdışı Rabat,Malzeme Tanımı ( EN ),Üretim Yeri,Key Cluster Kodu,Üst Sınıf Kodu,Ana Grup Kodu,VOR,Teps,Phase Out,Üretim yeri,Degisen Malzeme List
98683,RP311610,Parker F8UPMTB8-M12 Male Connect M12*1.5,RF-623-7232,NaN,NaN,0.0,0.0,0,0.0,0,I,NaN,NaN,0.01,0.0,0,0,42,PD,60,NaN,63,NaN,NaN,NaN,O,O,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Parker F8UPMTB8-M12 Male Connect M12*1.5,5000,NaN,NaN,NaN,NaN,NaN,NaN,5000 canter,None


In [33]:
df_envanter

,Malzeme Kodu,Malzeme Tanımı,Değişen Malzeme Kodu,Yerli Malzeme Kodu,Muadil Malzeme Kodu,Toplam Stok Miktarı,Kullanılabilir stok adedi,Back order adedi,"Açık, stok sip.adt.","Açık, acil sip.adt.",Tedarik türü,Tedarik yeri,Araç tipi,Standart maliyet,Ortalama maliyet,Yeni Sprş Seviyesi,Azami Stok,Mip Parti Byklğ,Mip Karakteristiği,Plnlı Tslmat Srsi,Sabitleme Srsi,Lead Time,Özel Tdrk,Blokaj,Parça Açıklaması,Replacibility Kod,Değişen Parça Replacibility,Orjinal Parca no,Orj. Parca Stok Ad.,Sipariş Kriteri,Cluster,Üst Sınıf,Ana Grup,Min. Order Qty.,Min. Package Qty.,Mevsimsel,Yurtiçi Rabat,Yurtdışı Rabat,Malzeme Tanımı ( EN ),Üretim Yeri,Key Cluster Kodu,Üst Sınıf Kodu,Ana Grup Kodu,VOR,Teps,Phase Out,Üretim yeri,Degisen Malzeme List
0,0123525501,ALTERNATÖR 100 A,0123525501Y,0123525501Y,NaN,0.0,0.0,0,0.0,0,I,BOSCH-TRY,OPALIN9 E4,337.49,0.00,0,0,42,PD,60,NaN,63,NaN,TB,NaN,NaN,O,NaN,0.0,P,ALTERNATÖR,MOTOR DETAYLARI,MOTOR,NaN,NaN,M3,7.0,D,"ALTERNATOR, 100 A",4000,Y0010,U001,G01,NaN,X,NaN,4000 canter,"['0123525501', '0123525501Y', '0123525507']"
1,0123525501Y,ALTERNATOR 100 A,0123525507,NaN,NaN,0.0,0.0,0,0.0,0,Y,BOSCH-TRY,OPALIN9 E5,0.01,0.00,0,0,42,PD,45,NaN,41,NaN,NaN,NaN,O,O,NaN,0.0,P,ALTERNATÖR,MOTOR DETAYLARI,MOTOR,NaN,NaN,M3,5.0,C,ALTERNATOR 100 A,4000,Y0010,U001,G01,NaN,X,NaN,4000 canter,"['0123525501', '0123525501Y', '0123525507']"
2,0123525507,ALTERNATOR 100 A,NaN,NaN,NaN,0.0,0.0,0,0.0,0,Y,BOSCH-TRY,METROP E4M,5497.60,0.00,0,0,42,PD,60,NaN,63,NaN,NaN,NaN,O,NaN,51.26101-7265,0.0,P,ALTERNATÖR,MOTOR DETAYLARI,MOTOR,NaN,NaN,M3,6.0,C,NaN,4000,Y0010,U001,G01,NaN,X,NaN,4000 canter,"['0123525501', '0123525501Y', '0123525507']"
3,0124525565,ALTERNATOR,NaN,NaN,NaN,0.0,0.0,0,1.0,0,Y,BOSCH-TRY,DIGER,8616.89,7290.58,0,0,42,PD,60,NaN,63,NaN,NaN,"* Otobüste Yaz dönemi sıkıntı yaşanan parçadır, dikkat!!",O,NaN,NaN,0.0,P,ALTERNATÖR,MOTOR DETAYLARI,MOTOR,NaN,NaN,M3,6.0,C,NaN,4000,Y0010,U001,G01,X,X,NaN,4000 canter,"['0124525216', '0124525565']"
4,0178-XC-11-14-01_A,DIN935 M14X2 A2 PASLANMAZ YARIKLI SOMUN,NaN,NaN,NaN,0.0,0.0,0,0.0,0,Y,ANADOL CİV,EQUAD,213.34,0.00,0,0,NaN,NaN,0,NaN,32,NaN,NaN,NaN,NaN,NaN,NaN,0.0,P,SOMUN,HARDWARE,DİĞER,NaN,NaN,NaN,NaN,L,DIN935 M14X2 A2 STAINLESS SLOT NUT,4000,Y0333,U112,G27,NaN,NaN,NaN,4000 canter,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116996,ZF-ECOFLUID M-02E,ŞANZIMAN YAĞ OTOMATİK,NaN,NaN,NaN,0.0,0.0,0,0.0,0,Y,TEMSANEW,SARF,0.00,0.00,0,0,42,PD,75,NaN,63,NaN,NaN,NaN,NaN,NaN,NaN,0.0,P,YAĞ,BAKIM PARÇALARI,BAKIM PAKETLERİ,NaN,NaN,M1,NaN,NaN,TRANSMISSION OIL AUTOMATIC,5000,Y0390,U154,G41,NaN,NaN,NaN,5000 canter,None
116997,ZIMBA TELI,"ZIMBA TELİ (39X26X36,5cm)",NaN,NaN,NaN,0.0,0.0,0,0.0,0,Y,EMRE END.,DIGER,1.00,0.00,0,0,42,PD,49,3.0,63,NaN,CA,NaN,NaN,NaN,NaN,0.0,P,SON 10 YIL İÇERİSİNDE HAREKET ETMEYEN,SON 10 YIL İÇERİSİNDE HAREKET ETMEYEN,DİĞER,NaN,NaN,M2,5,C,NaN,5000,Y0245,U169,G27,NaN,NaN,NaN,5000 canter,None
116998,ZIMBA.TELI,"ZIMBA TELİ (39X26X36,5cm)",NaN,NaN,NaN,0.0,0.0,0,0.0,0,Y,PIYASA,DIGER,190.00,270.00,0,0,42,PD,49,NaN,63,NaN,NaN,NaN,NaN,NaN,NaN,0.0,P,SARF,SARF,DİĞER,NaN,NaN,M1,4,L,STABLES,5000,Y0446,U122,G27,NaN,NaN,NaN,5000 canter,None
116999,ZR905Z,MOTOR SANTRIFUJ YAG FILTRESI,NaN,NaN,NaN,11.0,11.0,0,36.0,0,I,MANNHUMMEL,SAFIR P E5,585.95,471.36,0,0,42,PD,70,NaN,63,NaN,NaN,* Garantiden çıkan Safirlerin bakımında talep görüyor,NaN,NaN,D1922496,1.0,I,FİLTRE,MOTOR DETAYLARI,MOTOR,12,NaN,M3,4,B,ENGINE CENTRIFUGAL OIL FILTER,5000,Y0120,U001,G01,NaN,X,NaN,5000 canter,None


In [34]:
import numpy as np
df_envanter["Degisen Malzeme List"] = df_envanter["Degisen Malzeme List"].replace(["None", "none", "NaN", "nan", ""], np.nan)

In [35]:
df_envanter[df_envanter["Degisen Malzeme List"]=="None"]

,Malzeme Kodu,Malzeme Tanımı,Değişen Malzeme Kodu,Yerli Malzeme Kodu,Muadil Malzeme Kodu,Toplam Stok Miktarı,Kullanılabilir stok adedi,Back order adedi,"Açık, stok sip.adt.","Açık, acil sip.adt.",Tedarik türü,Tedarik yeri,Araç tipi,Standart maliyet,Ortalama maliyet,Yeni Sprş Seviyesi,Azami Stok,Mip Parti Byklğ,Mip Karakteristiği,Plnlı Tslmat Srsi,Sabitleme Srsi,Lead Time,Özel Tdrk,Blokaj,Parça Açıklaması,Replacibility Kod,Değişen Parça Replacibility,Orjinal Parca no,Orj. Parca Stok Ad.,Sipariş Kriteri,Cluster,Üst Sınıf,Ana Grup,Min. Order Qty.,Min. Package Qty.,Mevsimsel,Yurtiçi Rabat,Yurtdışı Rabat,Malzeme Tanımı ( EN ),Üretim Yeri,Key Cluster Kodu,Üst Sınıf Kodu,Ana Grup Kodu,VOR,Teps,Phase Out,Üretim yeri,Degisen Malzeme List


In [36]:
import ast

target_codes = ["RF-623-7232", "RP311610"]
mask = df_envanter["Malzeme Kodu"].isin(target_codes)

def add_targets(val):
    if pd.isna(val):
        return str(target_codes)
    try:
        current = ast.literal_eval(val) if isinstance(val, str) else val
    except Exception:
        current = []
    combined = current + target_codes
    return str(combined)

df_envanter.loc[mask, "Degisen Malzeme List"] = df_envanter.loc[mask, "Degisen Malzeme List"].apply(add_targets)

In [37]:
df_envanter[df_envanter["Malzeme Kodu"].isin(target_codes)]

,Malzeme Kodu,Malzeme Tanımı,Değişen Malzeme Kodu,Yerli Malzeme Kodu,Muadil Malzeme Kodu,Toplam Stok Miktarı,Kullanılabilir stok adedi,Back order adedi,"Açık, stok sip.adt.","Açık, acil sip.adt.",Tedarik türü,Tedarik yeri,Araç tipi,Standart maliyet,Ortalama maliyet,Yeni Sprş Seviyesi,Azami Stok,Mip Parti Byklğ,Mip Karakteristiği,Plnlı Tslmat Srsi,Sabitleme Srsi,Lead Time,Özel Tdrk,Blokaj,Parça Açıklaması,Replacibility Kod,Değişen Parça Replacibility,Orjinal Parca no,Orj. Parca Stok Ad.,Sipariş Kriteri,Cluster,Üst Sınıf,Ana Grup,Min. Order Qty.,Min. Package Qty.,Mevsimsel,Yurtiçi Rabat,Yurtdışı Rabat,Malzeme Tanımı ( EN ),Üretim Yeri,Key Cluster Kodu,Üst Sınıf Kodu,Ana Grup Kodu,VOR,Teps,Phase Out,Üretim yeri,Degisen Malzeme List
94435,RF-623-7232,"Raufoss Push-in male M12 / Tube 5/16""",RP311610,RF-623-7232Y,NaN,0.0,0.0,0,0.0,0,Y,TEMSANEW,TS35 EPA15,19.60,0.0,0,0,42,PD,70,3.0,63,NaN,TB,NaN,O,O,NaN,0.0,P,REKOR,HARDWARE,DİĞER,NaN,NaN,M1,6,E,"Raufoss Push-in male M12 / Tube 5/16""",5000,Y0308,U112,G27,NaN,NaN,NaN,5000 canter,"['RF-623-7232', 'RP311610']"
98683,RP311610,Parker F8UPMTB8-M12 Male Connect M12*1.5,RF-623-7232,NaN,NaN,0.0,0.0,0,0.0,0,I,NaN,NaN,0.01,0.0,0,0,42,PD,60,NaN,63,NaN,NaN,NaN,O,O,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Parker F8UPMTB8-M12 Male Connect M12*1.5,5000,NaN,NaN,NaN,NaN,NaN,NaN,5000 canter,"['RF-623-7232', 'RP311610']"


In [38]:
df_envanter["GuncelMalzeme"] = df_envanter["Degisen Malzeme List"].where(
    df_envanter["Degisen Malzeme List"].notna(),
    df_envanter["Malzeme Kodu"]
)

In [39]:
df_envanter[df_envanter["Malzeme Kodu"].isin(target_codes)]

,Malzeme Kodu,Malzeme Tanımı,Değişen Malzeme Kodu,Yerli Malzeme Kodu,Muadil Malzeme Kodu,Toplam Stok Miktarı,Kullanılabilir stok adedi,Back order adedi,"Açık, stok sip.adt.","Açık, acil sip.adt.",Tedarik türü,Tedarik yeri,Araç tipi,Standart maliyet,Ortalama maliyet,Yeni Sprş Seviyesi,Azami Stok,Mip Parti Byklğ,Mip Karakteristiği,Plnlı Tslmat Srsi,Sabitleme Srsi,Lead Time,Özel Tdrk,Blokaj,Parça Açıklaması,Replacibility Kod,Değişen Parça Replacibility,Orjinal Parca no,Orj. Parca Stok Ad.,Sipariş Kriteri,Cluster,Üst Sınıf,Ana Grup,Min. Order Qty.,Min. Package Qty.,Mevsimsel,Yurtiçi Rabat,Yurtdışı Rabat,Malzeme Tanımı ( EN ),Üretim Yeri,Key Cluster Kodu,Üst Sınıf Kodu,Ana Grup Kodu,VOR,Teps,Phase Out,Üretim yeri,Degisen Malzeme List,GuncelMalzeme
94435,RF-623-7232,"Raufoss Push-in male M12 / Tube 5/16""",RP311610,RF-623-7232Y,NaN,0.0,0.0,0,0.0,0,Y,TEMSANEW,TS35 EPA15,19.60,0.0,0,0,42,PD,70,3.0,63,NaN,TB,NaN,O,O,NaN,0.0,P,REKOR,HARDWARE,DİĞER,NaN,NaN,M1,6,E,"Raufoss Push-in male M12 / Tube 5/16""",5000,Y0308,U112,G27,NaN,NaN,NaN,5000 canter,"['RF-623-7232', 'RP311610']","['RF-623-7232', 'RP311610']"
98683,RP311610,Parker F8UPMTB8-M12 Male Connect M12*1.5,RF-623-7232,NaN,NaN,0.0,0.0,0,0.0,0,I,NaN,NaN,0.01,0.0,0,0,42,PD,60,NaN,63,NaN,NaN,NaN,O,O,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Parker F8UPMTB8-M12 Male Connect M12*1.5,5000,NaN,NaN,NaN,NaN,NaN,NaN,5000 canter,"['RF-623-7232', 'RP311610']","['RF-623-7232', 'RP311610']"


In [ ]:
df_envanter

In [ ]:
df_envanter.to_csv(r"csv_files/13_09_envanter_with_replacibility.csv", index=False)

#### Sipariş Verisine Uygulama

In [ ]:
df = pd.read_csv("31_08_teklif_iade_temiz_all_columns.csv")

In [ ]:
df

In [ ]:
import ast

zincir_map = {}
for lst in df_envanter["Degisen Malzeme List"].dropna():
    try:
        chain = ast.literal_eval(lst) if isinstance(lst, str) else lst
        for m in chain:
            zincir_map[m] = chain
    except:
        continue

df["GuncelMalzeme"] = df["Malzeme"].map(zincir_map).fillna(df["Malzeme"])

In [ ]:
df

In [ ]:
df["Ret nedeni"].unique()

In [ ]:
df["GuncelMalzeme"] = df["GuncelMalzeme"].astype(str)

In [ ]:
df["GuncelMalzeme"].nunique()

In [ ]:
df["Malzeme"].nunique()

In [ ]:
df.to_csv(r"csv_files/13_09_siparis_with_replacibility.csv", index=False)